# Logistic Regression_2


    
## Import GraphLab Create



In [1]:
import graphlab

## Carica il dataset delle review

In [ ]:
#https://www.kaggle.com/snap/amazon-fine-food-reviews file Reviews.csv 

In [6]:
products = graphlab.SFrame.read_csv('Reviews.csv')

Unable to parse line "95303,B00401OZ1U,ATO7WKMVJD6XQ,"Curcio ""\""",0,0,5,1335398400,Metromint Water,Wonderful tasting water.  For one that doesn't like to drink soda's or juices because of the high sugar content this product is a great addition to the healthy drinking.  Also p..."

1 lines failed to parse correctly

Finished parsing file /Users/matteoamadei/Desktop/classification/Reviews.csv

Parsing completed. Parsed 100 lines in 1.03788 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[int,str,str,str,int,int,int,int,str,str]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


...

Read 98051 lines. Lines per second: 66105.6

Unable to parse line "514285,B0081X097M,ATO7WKMVJD6XQ,"Curcio ""\""",0,0,5,1335398400,Metromint Water,Wonderful tasting water.  For one that doesn't like to drink soda's or juices because of the high sugar content this product is a great addition to the healthy drinking.  Also ..."

2 lines failed to parse correctly

Finished parsing file /Users/matteoamadei/Desktop/classification/Reviews.csv

Parsing completed. Parsed 568452 lines in 5.5095 secs.

In [3]:
def remove_punctuation(text):
    import string
    return text.translate(None, string.punctuation) 



In [4]:
products = products[products['Score'] != 3]
len(products)

525812

In [ ]:
products['sentiment'] = products['Score'].apply(lambda rating : +1 if rating > 3 else -1)
products

In [9]:
products['sentiment']

dtype: int
Rows: 525812
[1, -1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, -1, 1, 1, 1, 1, -1, -1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ... ]

In [116]:
products.head(10)['Summary']

dtype: str
Rows: 10
['Good Quality Dog Food', 'Not as Advertised', '"Delight" says it all', 'Cough Medicine', 'Great taffy', 'Nice Taffy', 'Great!  Just as good as the expensive brands!', 'Wonderful, tasty taffy', 'Yay Barley', 'Healthy Dog Food']

In [7]:
print '# of positive reviews =', len(products[products['sentiment']==1])
print '# of negative reviews =', len(products[products['sentiment']==-1])

# of positive reviews = 443775
# of negative reviews = 82037


Facciamo due semplici trasformazioni sui dati
1. Rimuoviamo la punteggiatura
2. Facciamo **word count**



In [8]:
important_words =["food", "one", "great", "love", "use", "like",
                  "seat", "get", "also", "really", "time", "bought",
                  "product", "good", "daughter", "much", "dog", "cat", "months", "put",
                  "still", "back", "used", "recommend", "first", "nice", "bag",
                  "two", "using", "got", "fit", "around", "diaper", "enough", "month", "price", 
                  "go", "could", "soft", "since", "buy", "room", "works", "made", "child", "keep",
                  "size", "need", "year", "big", "make", "take", "easily", "think", "crib",
                  "clean", "way", "quality", "thing", "better", "without", "set", "new", "every", 
                  "cute", "best", "bottles", "work", "purchased", "right", "lot", "side", "happy",
                  "comfortable", "toy", "able", "kids", "bit", "night", "long", "fits", "see", "us",
                  "another", "play", "day", "money", "monitor", "tried", "thought", "never", 
                  "item",
                  "hard", "plastic", "however", 
                  "reviews", "something", "going", "pump",
                  "bottle", "cup", "return", "amazon",
                  "different", "top", "want", "problem",
                  "know", "water", "try", "received", "sure", 
                  "times", "chair", "find", "hold", "gate", "open", 
                  "bottom", "away", "actually", "cheap", "worked", "getting",
                  "ordered", "came", "milk", "bad", "part", "worth", "found", 
                  "cover", "many", "design", "looking", "weeks", "say", "wanted", 
                  "look", "place", "purchase", "looks", "second", "piece", "box", 
                  "pretty", "trying", "difficult", "together", "though", "give", 
                  "started", "anything", "last", "company", "come", "returned", 
                  "maybe", "took", "broke", "makes", "stay", "instead", "idea", 
                  "head", "said", "went", "working", "high", "unit", "seems",
                   'easy', 'old', 'little', 'perfect', 'loves', 
                  'less', 'even', 'waste','disappointed','would','disgusting',
                  'yummy','cool','acid','beautiful','baked','chocolate','homemade', 'low-fat',
                  'nutty','pickled','plain','dry',
'delicious','delectable','delightful','fantastic','flavored','flavorless','tasteful',
                  'tasteless’,’tasty','tender','smelly','smells','rancid','small',
                  'fried','insipid','low','well']

In [9]:
products['review_clean'] = products['Text'].apply(remove_punctuation)

In [73]:
len(important_words)

211

In [10]:
for word in important_words:
    products[word] = products['review_clean'].apply(lambda s : s.split().count(word))

In [11]:
# Bilancio i due insiemi in modo da avere lo stesso numero di esempi positivi e negativi (ma potrei
#considerare anche il dataset di partenz

In [51]:
products_neg = products[products['sentiment'] == -1]
products_pos = products[products['sentiment'] == 1]


In [ ]:
                                                                                  )

In [49]:
prod_neg = products_neg[0:15000]

In [52]:
prod_pos = products_pos[0:15000]

In [5]:
#unisco i 15000 esempi positivi con quelli negativi

In [54]:
prod_red = prod_neg.append(prod_pos)

## Convertire SFrame in Numpy Array

Come visto per la regressione Numpy è una libreria molto potente per fare operazioni sulle matrici. 
Convertiamo i nostri dati in matrici e implementiamo i nostri algoritmi con le matrici. 

In [12]:
import numpy as np

Get Numpy prende le colonne da un Sframe e le converte in un array Numpy. 
Sono restituiti due array:
Uno che rappresenta le feature e un altro che rappresenta le etichette. Anche questa volta, come nel caso della regressione, avremo bisogno di una colonna addizionale intercept per tenere in considerazione l'intercetta. 

In [13]:
def get_numpy_data(data_sframe, features, label):
    data_sframe['intercept'] = 1
    features = ['intercept'] + features
    features_sframe = data_sframe[features]
    feature_matrix = features_sframe.to_numpy()
    label_sarray = data_sframe[label]
    label_array = label_sarray.to_numpy()
    return(feature_matrix, label_array)

In [55]:
# Convertiamo in numpy array prod red
feature_matrix, sentiment = get_numpy_data(prod_red, important_words, 'sentiment') 

In [56]:
feature_matrix.shape

(30000, 212)

In [22]:
len(sentiment)

30000

## Calcolare la probabilità condizionata

Dalla lezione precedente, la probabilità condizionata è data da 
$$
P(y_i = +1 | \mathbf{x}_i,\mathbf{w}) = \frac{1}{1 + \exp(-\mathbf{w}^T \phi(\mathbf{x}_i))},
$$

dove il vettore delle features $\phi(\mathbf{x}_i)$ rappresenta i word count delle **important words** nella review i-esima $\mathbf{x}_i$. 

In [17]:
'''
produce stima probabilistica per P(y_i = +1 | x_i, w).
nel range tra 0 e 1
'''
def predict_probability(feature_matrix, coefficients):
    # Fai il prodotto di feature_matrix e coefficienti
    scores = np.dot(feature_matrix, coefficients)
    
    # Calcola P(y_i = +1 | x_i, w) usando la link function
    predictions = map(lambda x: 1 / (1 + np.exp(- x)), scores)
    
    # ritorna predictions
    return predictions

 Come lavora con l'algebra matriciale

Dato che i word count sono memorizzati come colonne in **feature_matrix**,ogni i-esima riga della matrice corrisponde al feature vector $\phi(\mathbf{x}_i)$, relativa alla review i:
$$
[\text{feature_matrix}] =
\left[
\begin{array}{c}
\phi(\mathbf{x}_1)^T \\
\phi(\mathbf{x}_2)^T \\
\vdots \\
\phi(\mathbf{x}_N)^T
\end{array}
\right] =
\left[
\begin{array}{cccc}
\phi_0(\mathbf{x}_1) & \phi_1(\mathbf{x}_1) & \cdots & \phi_D(\mathbf{x}_1) \\
\phi_0(\mathbf{x}_2) & \phi_1(\mathbf{x}_2) & \cdots & h\phi_D(\mathbf{x}_2) \\
\vdots & \vdots & \ddots & \vdots \\
\phi_0(\mathbf{x}_N) & \phi_1(\mathbf{x}_N) & \cdots & \phi_D(\mathbf{x}_N)
\end{array}
\right]
$$


$$
[\text{score}] =
[\text{feature_matrix}]\mathbf{w} =
\left[
\begin{array}{c}
\phi(\mathbf{x}_1)^T \\
\phi(\mathbf{x}_2)^T \\
\vdots \\
\phi(\mathbf{x}_N)^T
\end{array}
\right]
\mathbf{w}
= \left[
\begin{array}{c}
\phi(\mathbf{x}_1)^T\mathbf{w} \\
\phi(\mathbf{x}_2)^T\mathbf{w} \\
\vdots \\
\phi(\mathbf{x}_N)^T\mathbf{w}
\end{array}
\right]
= \left[
\begin{array}{c}
\mathbf{w}^T h(\mathbf{x}_1) \\
\mathbf{w}^T h(\mathbf{x}_2) \\
\vdots \\
\mathbf{w}^T h(\mathbf{x}_N)
\end{array}
\right]
$$

**Checkpoint**
Facciamo alcuni esempi per comprendere se 
 `predict_probability` è implementata correttamente

In [77]:
dummy_feature_matrix = np.array([[1.,2.,3.], [1.,-1.,-1]])
dummy_coefficients = np.array([1., 3., -1.])

correct_scores      = np.array( [ 1.*1. + 2.*3. + 3.*(-1.),          1.*1. + (-1.)*3. + (-1.)*(-1.) ] )
correct_predictions = np.array( [ 1./(1+np.exp(-correct_scores[0])), 1./(1+np.exp(-correct_scores[1])) ] )

print 'I seguenti output devono corrispondere '
print '------------------------------------------------'
print 'predizione corretta           =', correct_predictions
print 'output di predict_probability =', predict_probability(dummy_feature_matrix, dummy_coefficients)

I seguenti output devono corrispondere 
------------------------------------------------
predizione corretta           = [ 0.98201379  0.26894142]
output di predict_probability = [0.98201379003790845, 0.2689414213699951]


## Calcolo la derivata di  log likelihood rispetto a un singolo coefficiente

$$
\frac{\partial\ell}{\partial w_j} = \sum_{i=1}^N \phi_j(\mathbf{x}_i)\left(\mathbf{1}[y_i = +1] - P(y_i = +1 | \mathbf{x}_i, \mathbf{w})\right)
$$

Scriviamo ora la funzione che calcola la derivata di log likelihood rispetto a un singolo coefficente  $w_j$. La funzione accetta due argomenti:
* `errors` un vettore contenente $\mathbf{1}[y_i = +1] - P(y_i = +1 | \mathbf{x}_i, \mathbf{w})$ per tutte le i $i$.
* `feature` un vettore contenente $\phi_j(\mathbf{x}_i)$  per tutte le $i$. 


In [18]:
def feature_derivative(errors, feature):     
    #Calcola il prodotto errors * feature
    derivative = np.dot(errors, feature)
    
    # ritorna la derivata 
    return derivative

In [19]:
def compute_log_likelihood(feature_matrix, sentiment, coefficients):
    indicator = (sentiment==+1)
    scores = np.dot(feature_matrix, coefficients)
     
    logexp = np.log(1. + np.exp(-scores))
    print logexp
    lp = np.sum((indicator-1)*scores - logexp)
    return lp

In [118]:
dummy_feature_matrix = np.array([[1.,2.,3.], [1.,-1.,-1]])
dummy_coefficients = np.array([1., 3., -1.])
dummy_sentiment = np.array([-1, 1])

correct_indicators  = np.array( [ -1==+1,                                       1==+1 ] )
correct_scores      = np.array( [ 1.*1. + 2.*3. + 3.*(-1.),                     1.*1. + (-1.)*3. + (-1.)*(-1.) ] )
correct_first_term  = np.array( [ (correct_indicators[0]-1)*correct_scores[0],  (correct_indicators[1]-1)*correct_scores[1] ] )
correct_second_term = np.array( [ np.log(1. + np.exp(-correct_scores[0])),      np.log(1. + np.exp(-correct_scores[1])) ] )

correct_ll          =      sum( [ correct_first_term[0]-correct_second_term[0], correct_first_term[1]-correct_second_term[1] ] ) 

print '------------------------------------------------'
print 'correct_log_likelihood           =', correct_ll
print 'output of compute_log_likelihood =', compute_log_likelihood(dummy_feature_matrix, dummy_sentiment, dummy_coefficients)

The following outputs must match 
------------------------------------------------
correct_log_likelihood           = -5.33141161544
output of compute_log_likelihood = [ 0.01814993  1.31326169]
-5.33141161544


## Ascesa del gradiente

In [20]:
from math import sqrt

def logistic_regression(feature_matrix, sentiment, initial_coefficients, step_size, tolerance):
    converged = False
    coefficients = np.array(initial_coefficients) # assicurati che sia un numpy array
    while not converged:
        # Predici P(y_i = +1|x_i,w) usando la funzione predict_probability() 
        predictions = predict_probability(feature_matrix, coefficients)
        
        # Calcola il valore della indicator function per (y_i = +1)
        indicator = (sentiment==+1)
        gradient_ascent =0
        # Calcola gli errori come indicator - predictions
        errors = indicator - predictions
        for j in xrange(len(coefficients)): # loop su ogni coefficiente (w0,w1...)
            
            # Ricorda che feature_matrix[:,j] è la colonna delle feature j
            # Calcola la derivata
            derivative = feature_derivative(errors, feature_matrix[:,j])
            
            #  aggiungi step size * derivata al coefficente corrente
            coefficients[j] += step_size * derivative
            
            gradient_ascent += derivative ** 2
        
        gradient_magnitude = sqrt(gradient_ascent)
        print gradient_magnitude
        if gradient_magnitude < tolerance:
            converged = True
    return coefficients

Ora eseguiamo il metodo logistic_regression

In [ ]:
coefficients = logistic_regression(feature_matrix, sentiment, initial_coefficients=np.zeros(212),
                                   step_size=7e-9, tolerance =1000)

3808.15522924
3808.02051026
3807.88581656
3807.75114812
3807.61650495
3807.48188704
3807.34729438
3807.21272696
3807.07818478
3806.94366783
3806.80917612
3806.67470962
3806.54026833
3806.40585225
3806.27146138
3806.1370957
3806.00275521
3805.8684399
3805.73414977
3805.59988481
3805.46564501
3805.33143037
3805.19724088
3805.06307654
3804.92893734
3804.79482327
3804.66073433
3804.52667051
3804.3926318
3804.25861821
3804.12462971
3803.99066631
3803.856728
3803.72281478
3803.58892663
3803.45506355
3803.32122554
3803.18741259
3803.05362469
3802.91986183
3802.78612402
3802.65241124
3802.51872349
3802.38506076
3802.25142305
3802.11781034
3801.98422264
3801.85065994
3801.71712223
3801.5836095
3801.45012175
3801.31665897
3801.18322116
3801.04980831
3800.91642041
3800.78305747
3800.64971946
3800.51640639
3800.38311824
3800.24985502
3800.11661672
3799.98340333
3799.85021484
3799.71705125
3799.58391255
3799.45079874
3799.31770981
3799.18464575
3799.05160656
3798.91859223
3798.78560276
3798.6526381

## Predire i sentimenti 

La classe per un'osservazione $\mathbf{x}$, come visto in precedenza, può essere calcolata a partire dai coefficienti  $\mathbf{w}$ usando la seguente formula: 
$$
\hat{y}_i = 
\left\{
\begin{array}{ll}
      +1 & \mathbf{x}_i^T\mathbf{w} > 0 \\
      -1 & \mathbf{x}_i^T\mathbf{w} \leq 0 \\
\end{array} 
\right.
$$

Calcoleremo le predictions in due steps:
* **Step 1**: Calcolo gli **scores** utilizzando **feature_matrix** e **coefficients**, attraverso il prodotto.
* **Step 2**: Usando la formula definita in precedenza, calcola la prediction a partire dagli scores.



In [88]:
# Calcola gli scores come prodotto tra feature_matrix e coefficienti.
scores = np.dot(feature_matrix, coefficients)
prod_red["scores"] = scores

 **Step 2** class_prediction

In [ ]:
y_hat = scores > 0


In [110]:
prod_red['sign']= y_hat

In [111]:
prod_red['sent_val'] = prod_red['sign'].apply(lambda rating : 1 if rating ==True else -1)



In [114]:
res =prod_red['sent_val']

In [123]:
corrette = (sentiment == res).sum() 
corrette

20434

## Calcolo dell'accuracy 


$$
\mbox{accuracy} = \frac{\mbox{# correctly classified data points}}{\mbox{# total data points}}
$$



In [122]:
corrette = (sentiment == res).sum() 
accuracy = float(corrette) / len(prod_red) 
print "-----------------------------------------------------"
print '# Reviews    classificate correttamente =',corrette
print '# Reviews non classificate correttamente  =', len(prod_red)-corrette
print '# Reviews totali                =', len(prod_red)
print "-----------------------------------------------------"
print 'Accuracy = %.3f' % accuracy

-----------------------------------------------------
# Reviews    classificate correttamente = 20434
# Reviews non classificate correttamente  = 9566
# Reviews totali                = 30000
-----------------------------------------------------
Accuracy = 0.681
